
# 🚗 Dynamic Pricing for Urban Parking Lots

This notebook implements **dynamic pricing** for urban parking lots using **Pathway**, **Pandas**, and **Bokeh**.

It streams real-time parking data and dynamically updates prices based on occupancy (and can be extended further).

---


In [1]:

!pip install pathway bokeh panel --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:

import numpy as np
import pandas as pd
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

pn.extension()



## 📄 Upload Your Dataset

👉 In Colab, use the left sidebar → Files → Upload → Select `dataset.csv`.  
Once uploaded, it will be available at `/content/dataset.csv`.


In [3]:

# Load the uploaded file
df = pd.read_csv('/content/dataset.csv')

# Combine Date and Time columns into one timestamp
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                 format='%d-%m-%Y %H:%M:%S')

# Sort by timestamp
df = df.sort_values('Timestamp').reset_index(drop=True)

# Save cleaned file for streaming
df[["Timestamp", "Occupancy", "Capacity"]].to_csv("parking_stream.csv", index=False)

df.head()


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00
1,5248,BHMNCPHST01,1200,26.140014,91.731000,237,bike,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
2,3936,BHMMBMMBX01,687,20.000035,78.000003,264,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
3,6560,BHMNCPNST01,485,26.140048,91.730972,249,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
4,17056,Shopping,1920,26.150504,91.733531,614,cycle,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00


In [4]:

class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int


In [5]:

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

fmt = "%Y-%m-%d %H:%M:%S"

data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)



## 💡 Pricing Logic

We start with a **demand-based price** using occupancy ratio.  
Later, you can extend this to include queue length, traffic, special events, or competitor prices.

---


In [6]:

BASE_PRICE = 10

def demand_based_price(occ_ratio):
    demand = occ_ratio  # Simple base demand score
    demand_norm = np.clip(demand, 0, 1)
    price = BASE_PRICE * (1 + 0.5 * demand_norm)
    price = np.clip(price, BASE_PRICE * 0.5, BASE_PRICE * 2)
    return price


In [10]:
import datetime

def clip_price(x):
    return max(BASE_PRICE * 0.5, min(x, BASE_PRICE * 2))

delta_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior(),
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
    )
    .with_columns(
        occ_ratio = pw.this.occ_max / pw.this.cap,
        raw_price = BASE_PRICE * (1 + 0.5 * pw.this.occ_max / pw.this.cap)
    )
    .with_columns(
        price = pw.apply(clip_price, pw.this.raw_price)
    )
)


In [11]:

def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Dynamic Parking Price (Daily)",
        x_axis_type="datetime",
    )
    fig.line("t", "price", source=source, line_width=2, color="navy", legend_label="Price")
    fig.circle("t", "price", source=source, size=6, color="red")
    fig.legend.location = "top_left"
    return fig

viz = delta_window.plot(price_plotter, sorting_col="t")

pn.Column(viz).servable()


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [12]:

%%capture --no-display
pw.run()


Output()


## 🏆 Model 3 Placeholder: Competitive Pricing

Here, you can add logic to factor in **nearby lot prices**, geographic distances, and rerouting suggestions.

- Calculate haversine distances.
- Compare competitor prices.
- Adjust or suggest reroute dynamically.

✅ You can implement this in future cells.



## 🏆 Model 3: Competitive Pricing Logic

In this model, we incorporate **competitor lot prices** and **geographic proximity** into the dynamic price calculation.

### 💡 Logic

- If your lot is **full and nearby lots are cheaper**, suggest rerouting or reduce price.
- If nearby lots are **more expensive**, you can increase price slightly.

We calculate approximate geographic proximity using **latitude and longitude** (Haversine formula).


In [ ]:

# Example: Adding competitor logic (simplified)

from math import radians, cos, sin, asin, sqrt

# Function to compute haversine distance (km)
def haversine(lat1, lon1, lat2, lon2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of earth in km
    return c * r

# Dummy example competitor lots
competitor_lots = pd.DataFrame({
    'Name': ['LotA', 'LotB'],
    'Latitude': [28.6129, 28.7041],
    'Longitude': [77.2295, 77.1025],
    'Price': [8, 12]
})

# Example current lot location
current_lat = 28.6448
current_lon = 77.2167

# Compute distances
competitor_lots['Distance'] = competitor_lots.apply(
    lambda row: haversine(current_lat, current_lon, row['Latitude'], row['Longitude']), axis=1
)

# Decision example logic
nearby_cheaper = competitor_lots[(competitor_lots['Distance'] < 5) & (competitor_lots['Price'] < BASE_PRICE)].any().any()

if nearby_cheaper:
    final_price = BASE_PRICE * 0.9  # Reduce price slightly to remain competitive
    suggestion = "Suggest rerouting some vehicles to cheaper nearby lots."
else:
    final_price = BASE_PRICE * 1.1  # Increase price since competition is more expensive
    suggestion = "Increase price slightly, as nearby lots are more expensive."

print(f"New price considering competitors: ${final_price:.2f}")
print(f"Suggestion: {suggestion}")
